# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster_Response_Database.db')
df = pd.read_sql("SELECT * FROM Disaster_Response_Database", engine)
y_cols=[]
for i in df.columns:
    if i not in ['id','message','original','genre']:
        y_cols.append(i)
df=df.dropna(subset=y_cols)
X = df['message']

y=df[y_cols]

In [3]:
print(X.shape)
X.isnull().sum()

(26039,)


0

In [4]:
print(y.shape)
y.isnull().sum()

(26039, 36)


related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

### 2. Write a tokenization function to process your text data

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

In [7]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove the stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]    
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier

pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),\
                     ('transformer',TfidfTransformer(smooth_idf=False)),\
                     ('clf',MultiOutputClassifier(KNeighborsClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.25,random_state=42)

In [10]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
from sklearn.metrics import classification_report
y_pred=pipeline.predict(X_test)      
y_pred

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [12]:
print(classification_report(y_test.values, y_pred, target_names=y_cols) ) 

                        precision    recall  f1-score   support

               related       0.76      0.92      0.83      4931
               request       0.35      0.02      0.04      1138
                 offer       0.00      0.00      0.00        32
           aid_related       0.32      0.01      0.02      2668
          medical_help       0.00      0.00      0.00       507
      medical_products       0.00      0.00      0.00       327
     search_and_rescue       0.00      0.00      0.00       200
              security       0.00      0.00      0.00       125
              military       0.00      0.00      0.00       197
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       412
                  food       0.00      0.00      0.00       726
               shelter       0.00      0.00      0.00       585
              clothing       0.00      0.00      0.00        93
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
parameters = {'clf__estimator__algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
              'clf__estimator__p': [1, 2]
             }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train,y_train)

In [ ]:
y_pred2=cv.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred2, target_names=y_cols) ) 

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.